In [ ]:
import pandas as pd
import re
import openai
import re
import requests
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity

API_KEY = ""
RESOURCE_ENDPOINT = "" 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
#Note: The openai-python library support for Azure OpenAI is in preview.
openai.api_version = "2023-05-15"



### Data generation (Skip this step if data is already generated )

In [ ]:
user_message = ""
def generate_titles():
    user_message =f""" 
    generate 100 titles of customer support articles in your industry, focusing on the areas of payroll and HR that support agent can look up to answer questions from customers who are employees with payroll managed by ADP.
    Output data into a list 
 
"""

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a customer support agent for ADP company"},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

titles = generate_titles()

In [ ]:
titles = titles.split("\n")

In [ ]:
titles[:10]

In [ ]:
import time
def generate_article(title):
    user_message =f""" 
    given this title of an article "{title}" that support agents can look up to answer questions from customers who are employees with payroll managed by ADP.
    Generate detail content of the article 
 
"""
    i=0
    while i<5:
        try:
            response = openai.ChatCompletion.create(
                engine="gpt-35-turbo", # engine = "deployment_name".
                messages=[
                    {"role": "system", "content": "You are a customer support agent for ADP company"},
                    {"role": "user", "content":user_message },
                ]
            )
            return response['choices'][0]['message']['content']
        except:
            i+=1
            time.sleep(5)
            

contents =[]
for title in titles:
    contents.append(generate_article(title))

In [ ]:
import os
folder_path ="../../../data/agent_assistant"
os.makedirs(folder_path, exist_ok=True)
i=0
articles ={}
for title, content in zip(titles, contents):
    file_name = f"article_{i}"
    file_content = title+"\n\n"+content

    with open(os.path.join(folder_path,file_name), 'w') as file:
        file.write(file_content)

    articles[file_name]= file_content
    i+=1



#Generate embeddings

In [ ]:
openai.api_version = "2022-12-01"

articles_emb = {article[0]:get_embedding(article[1], engine = 'text-embedding-ada-002')for article in list(articles.items())}

### Persist data

In [ ]:
import json

with open(os.path.join(folder_path,"articles_emb.json"), "w") as fp:
    json.dump(articles_emb,fp) 

with open(os.path.join(folder_path,"articles.json"), "w") as fp:
    json.dump(articles,fp) 


## Assistant Design

### Load data

In [ ]:
import json
import os
folder_path ="../../data/agent_assistant"
with open(os.path.join(folder_path, "articles_emb.json"), "r") as file:
    articles_emb = json.load(file)
with open(os.path.join(folder_path, "articles.json"), "r") as file:
    articles = json.load(file)

### 1. Tool to generate conversation

In [ ]:
openai.api_version = "2023-05-15"

def generate_conversation(streaming =False):
    user_message =f""" 
    Generate a phone conversation between an customer's employee and an ADP support agent in the area of HR and Payroll. The employee has question and the support agent tried to identity the problems and 
    and take time to use different search tool to come up with answer to the employee.
    Your response:
 
"""
    system_message = """
    You are a customer support agent for ADP company. 

    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ],
        stream=streaming
    )
    return response
            


### 2. Tool to extract problems from conversation. 

In [ ]:
openai.api_version = "2023-05-15"

user_message = ""
def extract_problems(conversation):
    user_message =f""" 
    Follow this on going conversation below and extract problems that each party may need help with and formulate the search query to the knowledge base search tool.
    <<conversattion>>
    {conversation}
    <<conversattion>>
    Output your response in JSON format {{"problem":"summary of problem", "search_query":"content of search query"}}
    There can be more than 1 problem(s)
    Output just JSON, nothing else.
    Your response:
 
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    You are given access to knowledge base search tool to find knowledge needed to find answer to questions. 

    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            
conversation=generate_conversation()['choices'][0]['message']['content']
problems=extract_problems(conversation)
problems

### 3. Tool to find article given a problem

##### Brute Force method

In [86]:
import numpy as np  
openai.api_version = "2022-12-01"

def find_article(question, topk=3):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    max_similarity = float('-inf')  
    best_file_name = None  
      
    # Compute cosine similarity between input vector and each label vector
    cosine_list=[]  
    for file_name, vector in articles_emb.items():  
        cosine_sim = cosine_similarity(input_vector, vector) 
        cosine_list.append((file_name,cosine_sim ))
    cosine_list.sort(key=lambda x:x[1],reverse=True)
    cosine_list= cosine_list[:topk]
    best_file_names = [file_name[0] for file_name in cosine_list]
    contents = [articles[best_file_name] for best_file_name in best_file_names]
    return best_file_names, contents
question = "When do I receive my W2 form?"
find_article(question)

(['article_3', 'article_0', 'article_7'],
 ["4. Understanding Your W-2: A Guide for ADP Payroll Customers \n\nIf you are an employee whose payroll is managed by ADP, you will receive a W-2 form at the end of each fiscal year. This form summarizes your earnings and taxes paid throughout the year, and is necessary for filing your tax returns.\n\nTo help you understand the information displayed on your W-2 form, we've created this guide:\n\n1. Box 1 - Wages, tips, and other compensation: This box includes all taxable income earned during the year, including bonuses, overtime pay, and vacation pay.\n\n2. Box 2 - Federal income tax withheld: This box shows the total amount of federal income tax withheld from your paycheck throughout the year. This amount is based on your W-4 form, which you fill out when you are hired.\n\n3. Box 3 - Social Security wages: This box shows the total amount of wages that are subject to Social Security taxes. This includes all taxable income up to the Social Sec

#### Vector Searh Lib (Faiss) method

In [87]:
import numpy as np  
import faiss

openai.api_version = "2022-12-01"
#Get the array of embeddings for all articles
file_names =[]
emb_vectors = []
for file_name, vector in articles_emb.items():  
    file_names.append(file_name)
    emb_vectors.append(vector)
emb_vectors = np.float32(emb_vectors)
faiss.normalize_L2(emb_vectors)

index = faiss.IndexFlatIP(len(vector)) 
index.add(emb_vectors)


def find_article_emb_vec(question, topk=3):  
    output_articles=[]
    output_contents =[]
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    input_vector = np.float32([input_vector])
    faiss.normalize_L2(input_vector)
    _,i = index.search(input_vector, k=topk)
    for idx in i[0]:
        output_articles.append(file_names[idx])
        output_contents.append(articles[file_names[idx]])

    return output_articles, output_contents
question = "When do I receive my W2 form?"
find_article_emb_vec(question,3)

(['article_3', 'article_0', 'article_7'],
 ["4. Understanding Your W-2: A Guide for ADP Payroll Customers \n\nIf you are an employee whose payroll is managed by ADP, you will receive a W-2 form at the end of each fiscal year. This form summarizes your earnings and taxes paid throughout the year, and is necessary for filing your tax returns.\n\nTo help you understand the information displayed on your W-2 form, we've created this guide:\n\n1. Box 1 - Wages, tips, and other compensation: This box includes all taxable income earned during the year, including bonuses, overtime pay, and vacation pay.\n\n2. Box 2 - Federal income tax withheld: This box shows the total amount of federal income tax withheld from your paycheck throughout the year. This amount is based on your W-4 form, which you fill out when you are hired.\n\n3. Box 3 - Social Security wages: This box shows the total amount of wages that are subject to Social Security taxes. This includes all taxable income up to the Social Sec

### 4. Tool to answer question from the given problem

In [ ]:
openai.api_version = "2023-03-15-preview"

def answer_assist(problem, search_query):

    articles, contents = find_article_emb_vec(search_query,2)
    articles_contents=""
    for article, content in zip(articles, contents):
        articles_contents += f""" 
        article:{article}
        content: {content}
    """
    articles_contents = f"""
    <<knowledge articles>>
    {articles_contents}
    <<knowledge articles>>
    """
    user_message =f""" 
    problem:{problem}
    {articles_contents}
    Your response:
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    Given the question or problem statement and the knowledge article you have, give the answer.
    Rely solely to the guidance from the article.If the knowlege article is not relavant to the question, say you don't know. Do not make up your answer. 
    Cite the name of the knowledge article as source for your answer.
    Format:
    Answer: your answer to the question
    Sources: [source1, source2]
    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content'], articles_contents

question = "When do I receive my W2 form?"
answer, articles_contents = answer_assist(question,question)
print(answer)
print("---------------content-----------------")

print(articles_contents)

### 5. Put tools all together

In [ ]:
def recommend(conversation):
    i=0
    while (i<5): #handle wrong format output
        problems=extract_problems(conversation)
        try:
            problems=json.loads(problems)
            print("problems", problems)
            for problem in problems:
                answer, articles_contents = answer_assist(problem['problem'],problem['search_query'])
                print(answer)
                print("---------------content-----------------")

                print(articles_contents)
            break

        except Exception as e:
            print(e)
            print("problem parsing json, problems string is ", problems)
            i+=1

        



### One time conversation 

In [ ]:
conversation =generate_conversation()['choices'][0]['message']['content']
print(f"Conversation {conversation}")
recommend(conversation)        

    

### Simulate a running conversation 

In [ ]:
import concurrent
openai.api_version = "2023-05-15"
import time
conversation_pause_duration=2
agent_assist_freq=5
conversation_buffer =[]
def stream_conversation(conversation_buffer, pause_duration=5):
    responses = generate_conversation(True)
    conversation_counter =0
    old_conversation_counter =0
    for response in responses:
        content = response['choices'][0]["delta"].get("content","")
        conversation_buffer.append(content) 
        if "\n"  not in content:
            print(content, end="")
        else:
            conversation_counter+=1
            if conversation_counter > old_conversation_counter+2:
                conversation = "".join(conversation_buffer)
                print("starting recommendation")
                print("conversation: ", conversation)
                recommend(conversation)
                print("ending recommendation")
                old_conversation_counter=conversation_counter



        
stream_conversation(conversation_buffer,conversation_pause_duration)